In [2]:
#importar librerias
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import requests
import re
from bs4 import BeautifulSoup
#Count words using dictionary
import urllib.request, urllib.parse, urllib.error
import re
from bs4 import BeautifulSoup
import ssl
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


In [3]:


def total(val,t):# buscar tesis dentro de una área de un repositorio
    html = urllib.request.urlopen(repo+val+"/recent-submissions").read()
    soup = BeautifulSoup(html, 'html.parser')
    pages = soup.find_all("a", {"class":"next-page-link"})
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    mayor = (max([int(item) for item in matches]) // 20) * 20#contabiliza el mayor valor de índice de página
    limites = list(range(0, mayor + 20, 20))
    for i in limites:
        html = urllib.request.urlopen(repo+val+"/recent-submissions?offset={}".format(i)).read()
        soup = BeautifulSoup(html, 'html.parser')
        j = soup.select("a:has(+span.Z3988)")#selecciona el link anterior que contiene esa clase
        for i in j:
            t+=1
            links_tesis["{}{}".format(key,t)] = i.get("href")#guarda el link obtenido 
    return t

def distinct(x,lista):#desmembrar lista de listas
    if len(lista) == 0:
        x.append(0)
    elif len(lista) >=2:
        x.append(",".join(lista))
    else:
        x.append(lista[0])
def extraccion(query, lista):#extracción de query y almacenamiento en lista
        linkeado = re.findall(query, html.decode())
        distinct(lista, linkeado)




In [112]:
#pucp
#extrae todos los links relacionados a infórmatica y estadística de licenciatura, bachillerato y maestría de la PUCP
url_licenciatura = "https://repositorio.pucp.edu.pe/index/handle/123456789/9119"
url_bachillerato = "https://repositorio.pucp.edu.pe/index/handle/123456789/135254"
url_maestria = "https://repositorio.pucp.edu.pe/index/handle/123456789/9043"
url_total = [url_bachillerato,url_maestria,url_licenciatura]
repo = "https://repositorio.pucp.edu.pe/"
links_tesis = dict()#link de tesis por carrera numerado
tesistas =[]
resumen =[]
fechas =[]
asesores = []
titulo = []
grado = []
especialidad = []
t = 0
f = 0
for i in url_total:
    links= dict() #links de carreras
    html = urllib.request.urlopen(i).read()
    soup = BeautifulSoup(html, 'html.parser')
    z = soup.find_all("a" , href = True)   
    for i in z:#buscar carreras relacionadas a estadística o infórmatica
        if i.find_all("span", string = re.compile("(?i)(estadística|Ingeniería Informática|Estadística informática|informática|informática .*)|.*Computación?.|.*Sistemas.*?")):
            links[i.text] = i.get("href")
    for key,val in links.items():#itera sobre todos los links de páginas de tesis
        index = total(val,t = f)       
        f =index 
    for key,val in links_tesis.items():#extracción de atributos por tesis
        html = urllib.request.urlopen(repo+val+"?show=full",context=ctx).read()
        #print(html)
        extraccion('<meta name="DC.creator" content="(.*)" />',tesistas) 
        extraccion('<meta name="DCTERMS.abstract" content="(.*)" xml:lang="es_.."',resumen) 
        extraccion('<title>(.*)</',titulo) 
        extraccion('<meta content="(.*)" name="citation_date">',fechas) 
        extraccion('<meta name="DC.contributor" content="(.*)" />',asesores) 
        extraccion('<a role="menuitem" href="/index/handle/.*">[1-9]. (.*)<',grado) 
        especialidad.append(key)
        
import pandas as pd
datos = {
   # 'Universidad' : uni,
    'Titulo': titulo,
    'Tesista': tesistas,
    'Grado' : grado,
    'Asesor' : asesores,
    'Resumen' : resumen,
    'Año' : fechas,
    "Especialidad": especialidad
}

df = pd.DataFrame(datos)
df.to_csv("PUCP_Tesis.csv",sep=";",index=False,encoding="utf-8")
a=pd.read_csv("PUCP_Tesis.csv",sep=";",encoding="utf-8",na_values=["0"])
a


{'Ciencias con mención en Estadística': '/index/handle/123456789/170993', 'Ciencias con mención en Ingeniería Informática': '/index/handle/123456789/170572'}
{'Estadística': '/index/handle/123456789/9075', 'Informática': '/index/handle/123456789/9090', 'Informática con mención en Ciencias de la Computación': '/index/handle/123456789/51445', 'Informática con mención en Ingeniería de Software': '/index/handle/123456789/51450'}
{'Estadística': '/index/handle/123456789/171497', 'Ingeniería Informática': '/index/handle/123456789/9139'}


,Titulo,Tesista,Grado,Asesor,Resumen,Año,Especialidad
0,Brecha en ingresos por g&eacute;nero en el sec...,"Ormeño Meza, Rubén Aaron",Bachillerato,"Sal y Rosas Celi, Víctor Giancarlo",La brecha salarial entre hombres y mujeres es ...,2021-04-26,Ciencias con mención en Estadística1
1,Redise&ntilde;o de productos de software sigui...,"Salinas Suazo, Elizabeth Karina",Bachillerato,"Paz Espinoza, Freddy Alberto,Cueva Moscoso, Rony",El presente trabajo de investigación muestra e...,2021-09-02,Ciencias con mención en Ingeniería Informática2
2,Revisi&oacute;n sistem&aacute;tica de las t&ea...,"Esparza Cabanillas, Patricia del Pilar",Bachillerato,"Paz Espinoza, Freddy Alberto,Cueva Moscoso, Rony",Los sistemas de software de punto de venta se ...,2021-08-25,Ciencias con mención en Ingeniería Informática3
3,Predicci&oacute;n temporal de calidad del aire...,"Paredes Salazar, Diego José",Bachillerato,"Villanueva Talavera, Edwin Rafael",El presente trabajo explora los estudios en lo...,2021-08-23,Ciencias con mención en Ingeniería Informática4
4,Desarrollo de un modelo de prestaci&oacute;n d...,"Pérez Cabrera, Estephany Elizabeth",Bachillerato,"Quispe Vilchez, Eder Ramiro",El presente trabajo de investigación presenta ...,2021-05-31,Ciencias con mención en Ingeniería Informática5
...,...,...,...,...,...,...,...
806,"An&aacute;lisis, dise&ntilde;o y construcci&oa...","Camarena Gil, Meylin Cinthia"" xml:lang=""es_ES,...",Licenciatura,NaN,En este trabajo de tesis se presenta el desarr...,2011-05-09,Ingeniería Informática578
807,"An&aacute;lisis, dise&ntilde;o e implementaci&...","Gutiérrez Zúñiga, Jessica"" xml:lang=""es_ES,Cha...",Licenciatura,NaN,El presente proyecto de tesis tiene por objeti...,2011-05-09,Ingeniería Informática579
808,An&aacute;lisis y dise&ntilde;o de una herrami...,"Valdiviezo Basauri, Manuel Javier"" xml:lang=""e...",Licenciatura,NaN,Para aquellos que desarrollan y mantienen los ...,2011-05-09,Ingeniería Informática580
809,Modelado ling&uuml;&iacute;stico-pros&oacute;d...,"Segura Salas, Elí Rónal"" xml:lang=""es_ES",Licenciatura,NaN,El objeto principal de esta tesis ha sido el e...,2011-05-09,Ingeniería Informática581
